# TensorFlow Basics
This Jupyter Notebook contains a very fast introduction to TensorFlow (TF).
This introduciton is **VERY** fast, and leaves many topics untouched or under touched, but should give you enough of an introduction to be able to follow the later notebooks.
If you want a deeper dive the following are good places to start:

#### External resources
* [Official getting started material](https://www.tensorflow.org/get_started/) - collection of good tutorials from beginer to very advanced
* **[Python API Guides](https://www.tensorflow.org/api_guides/python/array_ops)** - GREAT place to look up TF works! Has some descriptions of how and why to use different parts.
* [Documentation](https://www.tensorflow.org/api_docs/python/) - Short and consice descriptions of how everything in TF works.
* [LearningTensorFlow.org](http://learningtensorflow.com/getting_started/) - A website dedicated to teaching TF. They have some good tutorials at varying levels.


### What is TensorFlow
TensorFlow is a programming system in which you represent computations as graphs.
This graph is then compiled to efficient C code.
This added layer of abstraction makes it possible to have the same code run seamlessly on both GPU and parallel on CPU. 

TensorFlow provides multiple APIs. 
The lowest level API, **TensorFlow Core**, provides you with fine-grained control.
Higher level APIs, such as `tf.contrib.learn`, are built on top of TensorFlow Core, are generally faster and easier to use.
They help manage data, training, and inference.
This guide begins with an introduction to **TensorFlow Core**.
Later, in other exercises, we will demonstrate how to use Tensorflow in a way that is closer to how it is used in the real world.

**NB**: The some of the API whose names contain `contrib` are still in development, and their interface may change.

To use TensorFlow you need to understand how TensorFlow:
* Represents computations as graphs.
* Executes graphs in the context of Sessions.
* Represents data as tensors.
* Maintains state with Variables.
* Uses feeds and fetches to get data into and out of arbitrary operations.

**<span style="color:red">TODO</span>**:
Either make sure we cover all this, or tell which part we cover in this section!!




The two basic building blocks of TensorFlow are **tensors** and **operations** (called ops for short).
* **Tensors**: The edges in the graph
    * A Tensor is a typed multi-dimensional array, and are how information flows through the graph.
    * Tensors are used for data and parameters
* **Ops**: the nodes in the graph 
    * An op takes zero or more Tensors, performs some computation, and produces zero or more Tensors.

A TensorFlow graph is a description of computations. To compute anything, a graph must be launched in a `Session`. A Session places the graph ops onto `Devices`, such as CPUs or GPUs, and provides methods to execute them. These methods return tensors produced by ops as [numpy](http://www.numpy.org/) ndarray objects in Python, and as `tensorflow::Tensor` instances in C and C++.

TensorFlow can be used from C, C++, and Python programs, with Python being the most common and best supported.


# Example 1: The ultra basics


First we import TensorFlow, and some other handy libraries.

In [ ]:
## Import libraries
import tensorflow as tf
import numpy as np
import os
import sys
sys.path.append(os.path.join('.', '..')) # Allow us to import shared custom 
                                         # libraries, like utils.py
import utils # contain various helper funcitons that aren't 
             # important to understand

### Basic operations

Let us begin with a simple example -- 2D linear regression: $y = ax + b$. Where $x$ is the input, $y$ is the output, and $a,~b$ are the parameters.

For starters let us compute $y$ when $a=2$, and $b=-1$ for a couple of different $x$.

In [ ]:
## Building the computational graph

# In case we have already created something: clear it
tf.reset_default_graph()

# Create the two variables
# The 'name' argument indicates what TF should call the variable internally.
# It is a good idea to properly name your ops, as it makes debugging and later analysis much easier!
a = tf.Variable(2., name="a")
b = tf.Variable(-1., name="b")

# Create an x variable, with some numbers
x_values = [-2, -1, 0, 1, 2]
x_var = tf.Variable(x_values, name="xVariable", dtype=tf.float32)

# Define y
with tf.name_scope('yFromVariable'): # Give y a name
    y_var = a*x_var + b

print(y_var)

#### What just happened?
Why didn't `print(y_var)` print out the answer?

You might have expected `y_var` to be the results of `a*x_var + b`, but instead we got an **Tensor** object.
This is because before TensorFlow works by first creating a computational graph representation.
This graph can then be compiled, which can then be run.
This means that `y_var` is a TF **operation**, i.e. a node in the computational graph.

Lets compile and run `y_var` now!

In [ ]:
# Create an operation that will initialize the graph.
init_op = tf.global_variables_initializer()

# TensorFlow operations are performed by 'Sessions'
with tf.Session() as sess:
    # Run the operation that initializes the graph. 
    # We almost always do this as the first thing.
    sess.run(init_op)
    
    # Compute y by running the operation
    y_output = sess.run(y_var)

print('y_output is a ' + str(type(y_output)) + '\n')

# Print the results
print('{:4s}  {:4s}'.format('x_var', '  y'))
for i in range(len(x_values)):
    s = "{:4.1f} : {:4.1f}"
    print(s.format(x_values[i], y_output[i]))


That is all well an good, but what if we wanted compute this for another $x$?
Right now we have defined `x_var` as a `tf.variable`.
This makes changing it cumbersome.
A better approach is using `tf.placeholder`, which we will do now.

A `placeholder` has 3 important arguments:
* **`dtype`** specifying what kind of data we are dealing with. Generally use **tf.float32**, as most GPU's are only optimized for 32 bit floating points
* **`shape`** lets TF know the dimensions of the variable. Writing `None` allows us to change the number of dimensions, without having to recompile the graph. This however prevents some optimization, so it should be specified when possible.
* **`name`** is what TF will call the placeholder internally. For instance this is the name it will use if the placeholder causes an error.

In [ ]:
# Create x as a placeholder now!
x_ph = tf.placeholder(dtype=tf.float32, shape=[None], name="xPlaceholder")

# Define another y, using the placeholder x this time
with tf.name_scope('yFromPlaceholder'):
    y_ph = a*x_ph + b

x_new_values = [-0.2, -0.1, 0, 0.1, 0.2]

## Compute y
with tf.Session() as sess:
    sess.run(init_op)
    feed_dict = {x_ph : x_new_values}
    y_output = sess.run(y_ph, feed_dict=feed_dict)

    
# Print the results
print('{:4s}  {:4s}'.format(' x_ph', '  y'))
for i in range(len(x_values)):
    s = "{:4.1f} : {:4.1f}"
    print(s.format(x_new_values[i], y_output[i]))

#### What just happened?

This time, when we created the graph, we created $x$ as a `tf.placeholder`.
This means that `x_ph` simply stands in the place of real data.
So when we want to compute `y` for a particular value we simply **feed** that value into the graph, using a `feed_dict`.

If we wanted to change the values now, we simply need to feed a new value. 

These **variables** and **palceholders** can be a little hard to wrap your head around at first.
It can therefore be a good idea to read up on these:

#### External resources:
* **Variables**: 
    [Guide](https://www.tensorflow.org/programmers_guide/variables),
    [Documentation](https://www.tensorflow.org/api_docs/python/tf/Variable)
* **Placeholders**: 
    [Documentation](https://www.tensorflow.org/versions/r0.11/api_docs/python/io_ops/placeholders)

## Examining the graph with TensorBoard

Normally TensorBoard is run separately by typing in the command prompt:

    tensorboard logdir==<path/to/TensorBoard/logs

And then accessed by going to the correct port. Typically `localhost:6006` in your browser of choice.

In this notebook however we will show it in-line however.
If you want to see how to launch TensorBoard normally see the **`demo_Launching_TensorBoard.ipynb`** notebook, located in this folder.

When you execute the cell below you should see a graph that represents the work we have done so far.

Click a node to see its attributes and high level information.
**Double click a node to expand it**. Try double clicking on one of the y's.
Doing so will show you the operations that are necessary to compute $y$, i.e. a multiplication and an addition.
This is especially useful for examining the dimensions of your data as it flows through the graph.

The TensorBoard graph visualizer is a great tool for examining your model.
It is important to use `tf.name_scope` to dutifully name your variables properly!
Otherwise the graph visualizer quickly becomes unwieldy and useless.
This takes practice, but it is well worth it.
Propper usage of `tf.name_scope` also makes debugging easier, so it is a good habbit to get into.

If you are interested in how to embed TensorBoard in the notebook see the `../utils.py` file.
There are many details that we didn't cover here, but here is a good place to start:

#### External resources
* **TensorBoard**: 
    [Graph visualization](https://www.tensorflow.org/get_started/graph_viz), 
    [Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard), 
    [Embedding Visualization](https://www.tensorflow.org/get_started/embedding_viz).

In [ ]:
## Launch TensorBoard, and visualize the TF graph
tmp_def = utils.rename_nodes(sess.graph_def, lambda s:"/".join(s.split('_',1)))
utils.show_graph(tmp_def)

**Mini-assignment**: 
Try and change the argument in `tf.name_scope` when defining `y_var` and `y_ph` to `y_variable`, and `y_placeholder`.
Then run code visualizing TensorBoard again.
* Notice what changed? Can you think of when this kind of thing is smart to do?

## <span style="color:red"> Assignemnt 1: Your first TensorFlow op</span>
For the first assignment you must implement Pythagoras' famous equation:
$$c = \sqrt{d^2 + e^2}$$

You should create $a$ and $b$ as placeholders, and then compute $c$ for $d = {3, 2, 1}$ and $e = {4,5,6}$.

It is **important** that you use TF ops for all the computations on the graph.
(e.g. use `tf.square` instead of `np.square`)
Otherwise TF can't optimize the code properly, and you risk it becoming VERY slow.
You can find the TF math ops that you need [here](https://www.tensorflow.org/versions/r0.11/api_docs/python/math_ops/basic_math_functions).

In [ ]:
e_values = [3,2,1]
d_values = [4,5,6]

## Your code here!
# 1) Define the placeholders and the graph

# 2) Start a session, and compute the output
c_output = [0, 0, 0] # Use this variable name as your output



In [ ]:
# Print out the results, and validate you get the correct values
true_values = np.sqrt(np.square(e_values) + np.square(d_values))
assingment_1_success = True

for i in range(len(c_output)):
    assingment_1_success = False if not np.abs(true_values[i] - c_output[i]) < 1e-6 else assingment_1_success
    print('Corect value {:4.3f}, your value {:4.3f}. '.format(true_values[i], c_output[i]), end='')
    if not assingment_1_success: 
        print("Oops :(")
        print("\nSometihng went wrong, and the output isn't as expected.\
               \nGo back and have a look, or ask someone for help.")
        break
    print('Correct!')

    
if assingment_1_success:
    print('\nGood job! \nTake a break, strecht your legs, and then continue onwards!')
    

**Mini-assignment**:
After having successfully completed the assignment go back and run the code TensorBoard visualization code again.
* Did you remember to give your variables and placeholders meaningful names, or does everything look like a mess?

# Example 2: Linear Regression

Content
* How to update the parameters using gradient descent
* How to use TensorBoard to visualize the training



# Credits
Created by Toke Faurby ([faur](https://github.com/Faur)), based on previous work by 
* Lars Maaløe ([larsmaaloee](https://github.com/larsmaaloee))
* Casper Sønderby ([casperkaae](https://github.com/casperkaae))
* Søren Kaae Sønderby ([skaae](https://github.com/skaae))
* Jonas Busk ([jonasbusk](https://github.com/jonasbusk))
* Alexander R Johansen ([alrojo](https://github.com/alrojo))
